Repovizz2 DataPack Upload Example
=================================

This notebook demonstrates basic datapack operations (upload and modify)

We will use here the repovizz2 client library to ease the authentication, but it is basically a standard OAuth2 authentication.

In [1]:
from repovizz2 import RepoVizzClient

We will be using those modules:

In [2]:
import json
# This is for the notebook
from IPython.display import IFrame, display

Now, some global parameters to connect to repovizz2

In [3]:
CLIENT_ID = "4991097c-9a0c-4a56-a337-2b2177c5aa74"
CLIENT_SECRET = "97c055e7-5dd1-49e8-a6da-126d58f07ce1"


Repovizz2 authentication
-----------------------

We authenticate the client (this script) against repoVizz2 using a registered user. For more details on the authentication system see [repoVizz2 authentication](Authentication.ipynb).


This will create an iframe in the notebook to authenticate the client, but you can open a new browser to do so, if you want

In [4]:
repovizz_client = RepoVizzClient(client_id=CLIENT_ID,client_secret=CLIENT_SECRET, repovizz_url="http://localhost:8080")
authorization_url = repovizz_client.start_auth()
repovizz_client.start_server(async=True)

# print authorization_url # and click on it
# - or -
# import webbrowser
# webbrowser.open(authorization_url)
# - or -
display(IFrame(authorization_url,800,600))

In [5]:
repovizz_client.finish_auth()

We first get info about our identity. We can use `/api/v1.0/user` to get json information about the current logged user.

In [6]:
myself = repovizz_client.get("/api/v1.0/user")
username = myself['username']
userid = myself['id']

Now we upload the datapack (without the data). The result of the call will include the upload datapack (item), including its ID. To do so, we post to `/api/v1.0/datapacks` with a new datapack using json.

In [7]:
datapack_json = json.load(open('MYO_example_datapack/MYO_example_datapack.json'))
result = repovizz_client.post(
    "/api/v1.0/datapacks",
    json={
        'structure': datapack_json,
        'name': datapack_json['info']['name'],
        'owner': userid,
    },
)
datapack = result['item']

Datapacks usually refer to external material, such as `csv` files, videos or audio. Those were'nt uploaded to the server yet. Instead, reference to their filenames were used as handles.

To upload all the real assets, first we can ask for all the data files referenced in the datapack using `/api/v1.0/datapacks/{datapack_id}/content`. This returns us a list of the referenced filenames.

In [8]:
datapack_id = datapack['id']
data_in_datapack = repovizz_client.get('/api/v1.0/datapacks/{}/content'.format(datapack_id)).keys()
data_in_datapack

[u'orientation.csv',
 u'gyroscope.csv',
 u'video.mp4',
 u'accelerometer.csv',
 u'emg.csv',
 u'audio.wav']

For each of those files, we have to send the actual content to be saved in the server. We can do that by posting to `/api/v1.0/datapacks/{datapack_id}/content/{content_id}` with the actual file attached.

Repovizz2 will substitute the content identifier with a new one, using the format `repovizz:{hashcode}`, meaning that the contents of this file can be found in the repovizz2 server, with the _hashcode_ as its handle.

In [9]:
for fname in data_in_datapack:
    print repovizz_client.post(
        '/api/v1.0/datapacks/{}/content/{}'.format(datapack_id,fname),
        files={fname:open('MYO_example_datapack/'+fname)}
    )


{u'id': u'd657a300c1655c471a4d0517a348ba2dcae6e967'}
{u'id': u'c15b2a61444b9805e781b6a7194ab21df699677e'}
{u'id': u'9ec009d2ea38b6d600f793424739dcdc75bf8a7c'}
{u'id': u'095735930d80c313dfb29cdb558645d44d89269e'}
{u'id': u'a7bd73a0316c56f79e664b1901025be1f3db3f02'}
{u'id': u'a910ba389ca2847b6bf548fadad68ccebeee4f54'}


We can now check that indeed we changed the references to the contents in the datapack by quering again `/api/v1.0/datapacks/{datapack_id}/content`.

In [10]:
data_in_datapack = repovizz_client.get('/api/v1.0/datapacks/{}/content'.format(datapack_id)).keys()
data_in_datapack

[u'repovizz:9ec009d2ea38b6d600f793424739dcdc75bf8a7c',
 u'repovizz:a7bd73a0316c56f79e664b1901025be1f3db3f02',
 u'repovizz:095735930d80c313dfb29cdb558645d44d89269e',
 u'repovizz:d657a300c1655c471a4d0517a348ba2dcae6e967',
 u'repovizz:c15b2a61444b9805e781b6a7194ab21df699677e',
 u'repovizz:a910ba389ca2847b6bf548fadad68ccebeee4f54']

With this step completed, we concluded uploading the datapack to repovizz2.